In [1]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import pandas as pd
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

import tensorflow_hub as hub
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split


max_words = 15000
tokenizer = Tokenizer(num_words=max_words)
maxlen = 150




In [2]:
module_url = "https://tfhub.dev/google/elmo/3"
embed = hub.load("/Users/shantanu/elmo_3")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
root_dir = r'/Users/shantanu/Library/Mobile Documents/com~apple~CloudDocs/Capstone/Wikipedia Data'

result_path = annot_file_path = os.path.join(root_dir , 'comments_with_grouped_annoptations.tsv')

merged_comments = pd.read_table(result_path)
merged_comments['recipient_attack'] = merged_comments['recipient_attack'].apply(lambda x : 1 if x> 1 else 0 )
X_train = merged_comments['comment']

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)


data = pad_sequences(sequences, maxlen=maxlen)
print('Shape of data tensor:', data.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
y = merged_comments['recipient_attack']

one_hot_train_labels = to_categorical(y)

labels = np.asarray(one_hot_train_labels)
print('Shape of label tensor:', labels.shape)
labels = labels[indices]


X_train, X_test_validate, y_train, y_test_validate = train_test_split(merged_comments.comment, merged_comments.recipient_attack,
                                                    stratify=labels,
                                                    test_size=0.995,
                                                    random_state=123)

DF_train = X_train.to_frame()
DF_train['recipient_attack'] = y_train

DF_train.reset_index(inplace=True)

Shape of data tensor: (115864, 150)
Shape of label tensor: (115864, 2)


In [16]:
import pickle
with open("../tokenizer.pkl","rb") as f:
    tokenizer = pickle.load(f)

In [17]:
### Model load 

model = load_model('../basic_lstm_with_auto_encoder.h5')


In [18]:
import time
import random
import os


def preprocess_text(text_eval):
    sequences = tokenizer.texts_to_sequences([text_eval])
    data = pad_sequences(sequences, maxlen=maxlen)
    return data


def abalation(args  ):
    
    pred_vals_array = np.empty((0,2), float)
    NAE_vals_array = np.empty((3,1024), float)
    PAE_vals_array = np.empty((3,1024), float)

    from tensorflow.keras.models import load_model as lm1
    index, row = args
    print("running for {}".format(index))


    text = row.comment.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens_without_sw = tokens #[word for word in tokens if not word in stopwords.words()]
    

    ### First Get the overall Classification

#     print(tokens_without_sw)
    
    tokens_string = str(tokens_without_sw)
    
    prep_text = preprocess_text(tokens_string)
    
    pred_vlaues = model.predict(prep_text)

    classification = pred_vlaues.argmax()
    
    if classification == 0 and row.recipient_attack == 0:
        mode = 'PAE'  # Positive AutoEncoder
    elif classification == 0 and row.recipient_attack == 1:
        mode = 'NAE'
    elif classification == 1 and row.recipient_attack == 0:
        mode = 'PAE'
    elif classification == 1 and row.recipient_attack == 1:
        mode = 'NAE'

    ### Get the Elmo vector
    
    text = tf.convert_to_tensor([str(tokens_without_sw)])
    out = embed.signatures['default'](text)['elmo']

    pred_vals_array = np.empty((0, 2), float)
    
    for i in range(len(tokens_without_sw) - 2):
        
        text_eval = tokens_without_sw[i:i + 3]
        
        
        a = preprocess_text(text_eval)
        
    
        pred_vlaues = model.predict(a)
    
          
        pred_vals_array = np.vstack((pred_vals_array, pred_vlaues))
        
        
    
    if mode == "NAE":

        # Only Negative Vals, and find the index of the highest contributing phrase

        phrase_start_index = pred_vals_array[:, 1].argmax()

        ### Get the ELMO Encoding of the entire vector:

        elmo_vecs_NAE = np.array(out[0][phrase_start_index:phrase_start_index + 3])

        
        NAE_vals_array = np.vstack((NAE_vals_array, elmo_vecs_NAE))
        
#         final_list_NAE.append(elmo_vecs_NAE)

    elif mode == "PAE":

        # Only Positive Vals, and find the index of the highest contributing phrase

        phrase_start_index = pred_vals_array[:, 0].argmax()
        
        elmo_vecs_PAE = out[0][phrase_start_index:phrase_start_index + 3]

        PAE_vals_array = np.vstack((PAE_vals_array, elmo_vecs_PAE))
        
#         final_list_PAE.append(elmo_vecs_PAE)


In [19]:
DF_train.tail()

,index,comment,recipient_attack
574,6443,CategoryNEWLINE_TOKENNEWLINE_TOKENIt has been ...,0
575,97387,NEWLINE_TOKENNEWLINE_TOKENThat is fucking ridi...,1
576,64575,Extended comments about thisNEWLINE_TOKENThe c...,0
577,37597,`NEWLINE_TOKENNEWLINE_TOKENYou're showing sign...,0
578,10037,NEWLINE_TOKENNEWLINE_TOKENLatest press reports...,0


In [20]:
import tensorflow as tf

In [21]:
tf.__version__

'2.0.0'

In [22]:
tf.data.Dataset 

tensorflow.python.data.ops.dataset_ops.DatasetV2

In [24]:
# tf.data.Dataset()
# .map(
#     a,
#     num_parallel_calls=tf.data.experimental.AUTOTUNE
# )


In [9]:
def run_sequential ():
    for rec in DF_train.iterrows():
        abalation(rec)


In [3]:
# run_sequential()